In [1]:
import torch, math
from HTorch.manifolds import Euclidean, PoincareBall, Lorentz, HalfSpace, Sphere
from torch.nn import Parameter
from HTorch.utils.math_utils import arcosh, cosh, sinh, sq_norm, inner_product

L = Lorentz()
B = PoincareBall()
U = HalfSpace()

## Lorentz to Poincare (correct)

In [2]:
torch.manual_seed(0)
c = 1.2
#L to B
def to_poincare_tan(x, v, c):
    sqrt_c = c ** 0.5
    x_n = x[..., -1].unsqueeze(-1)
    v_n = v[..., -1].unsqueeze(-1)
    tmp = sqrt_c*x_n + 1
    comp_1 = v[..., :-1]/tmp 
    comp_2 = ((sqrt_c*v_n)/tmp.square())*x[..., :-1]
    return comp_1-comp_2

# randomly init x 
xl = L.proj(torch.randn(21, 5, 11, dtype=torch.float64), c=c)
assert L.check(xl, c) == True
xl_tan = L.proj_tan(xl, torch.randn_like(xl), c)
assert L.check_tan(xl, xl_tan, c) == True
# get y through expmap
yl = L.expmap(xl, xl_tan, c)
# get tan in poincare
xp_tan = to_poincare_tan(xl, xl_tan, c)
xp_tan_prime = L.to_poincare_tan(xl, xl_tan, c)
xp = L.to_poincare(xl, c)
# yp transform from yl
yp_from_L = L.to_poincare(yl, c)
yp_from_expmap = B.expmap(xp, xp_tan, c)
assert torch.all(torch.isclose(yp_from_L, yp_from_expmap))
yp_from_L[0,0], yp_from_expmap[0,0], B.check(yp_from_expmap, c)

(tensor([-0.4984,  0.0444, -0.1788,  0.2044, -0.1994, -0.3631, -0.1570, -0.2074,
         -0.2915,  0.3771], dtype=torch.float64),
 tensor([-0.4984,  0.0444, -0.1788,  0.2044, -0.1994, -0.3631, -0.1570, -0.2074,
         -0.2915,  0.3771], dtype=torch.float64),
 tensor(True))

## Lorentz to Halfspace (correct)

In [2]:
torch.manual_seed(0)
c = 1.2
#L to U
def to_halfspace_tan(x, v, c):
    sqrt_c = c ** 0.5
    x_i = x[..., :-2]
    v_i = v[..., :-2]
    # x_n - x_n-1
    x_diff = (x[..., -1] - x[..., -2]).unsqueeze(-1)
    sq_x_diff = x_diff.square()
    # v_n - v_n-1
    v_diff = (v[..., -2] - v[..., -1]).unsqueeze(-1)
    u = torch.zeros_like(x[..., :-1])
    # u_i
    u[..., :-1] = v_i/(sqrt_c*x_diff) + (v_diff/(sqrt_c*sq_x_diff))*x_i
    # u_n
    u[..., -1] = (v_diff/(c*sq_x_diff)).squeeze(-1)
    return u

# randomly init x 
xl = L.proj(torch.randn(10, 5, 21, dtype=torch.float64), c=c)
assert L.check(xl, c) == True
xl_tan = L.proj_tan(xl, torch.randn_like(xl), c)
assert L.check_tan(xl, xl_tan, c) == True
# get y through expmap
yl = L.expmap(xl, xl_tan, c)
# get tan in halfspace
xh_tan = to_halfspace_tan(xl, xl_tan, c)
xh_tan_prime = L.to_halfspace_tan(xl, xl_tan, c)
xh = L.to_halfspace(xl, c)
# yh transform from yl
yh_from_L = L.to_halfspace(yl, c)
yh_from_expmap = U.expmap(xh, xh_tan, c)
assert torch.all(torch.isclose(yh_from_L, yh_from_expmap))
yh_from_L[0,0]-yh_from_expmap[0,0], U.check(yh_from_expmap, c)

(tensor([ 0.0000e+00,  1.3878e-17,  2.7756e-17,  2.7756e-17,  2.7756e-17,
         -2.7756e-17,  0.0000e+00,  0.0000e+00,  5.5511e-17,  0.0000e+00,
         -6.9389e-18,  0.0000e+00,  0.0000e+00,  2.7756e-17, -1.7347e-18,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.9031e-18],
        dtype=torch.float64),
 tensor(True))

In [3]:
xh_tan.size(), xh.size()

(torch.Size([10, 5, 20]), torch.Size([10, 5, 20]))

In [6]:
L.norm_t(xl_tan, x=xl, c=c) - U.norm_t(xh_tan, x=xh, c=c)

torch.Size([10, 5, 1]) torch.Size([10, 5, 1])


tensor([[[ 0.0000e+00],
         [ 8.8818e-16],
         [-1.7764e-15],
         [ 0.0000e+00],
         [-8.8818e-16]],

        [[ 0.0000e+00],
         [-8.8818e-16],
         [-8.8818e-16],
         [-8.8818e-16],
         [ 0.0000e+00]],

        [[-4.4409e-16],
         [-4.4409e-16],
         [ 0.0000e+00],
         [ 4.4409e-16],
         [ 0.0000e+00]],

        [[ 4.4409e-16],
         [ 4.4409e-16],
         [-1.7764e-15],
         [-1.7764e-15],
         [-8.8818e-16]],

        [[ 0.0000e+00],
         [-4.4409e-16],
         [ 8.8818e-16],
         [-8.8818e-16],
         [-8.8818e-16]],

        [[-8.8818e-16],
         [-4.4409e-16],
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 8.8818e-16]],

        [[ 0.0000e+00],
         [-8.8818e-16],
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00]],

        [[-4.4409e-16],
         [ 0.0000e+00],
         [ 8.8818e-16],
         [ 0.0000e+00],
         [-8.8818e-16]],

        [[-8.8818e-16],


## Halfspace to Lorentz (correct)

In [4]:
torch.manual_seed(0)
c = 1.2
# U to L; halfspace to lorentz
############################################
def to_lorentz_tan(x, v, c):
    # sum of xj^2 where j=1 to n-1
    sqrt_c = c**0.5
    xi = x[..., :-1]
    vi = v[..., :-1]
    xn = x[..., -1].unsqueeze(-1)
    sq_xn = xn.square()
    sq_norm_xi = sq_norm(xi)
    vn = v[..., -1].unsqueeze(-1)

    other_comps =  (1/(sqrt_c*xn))*vi - vn*xi/(sqrt_c*sq_xn) 
    inv_cx2n = 1/(c*sq_xn)
    tail_comp_tmp1 = inner_product(xi, vi)/xn
    tail_comp_tmp2 =  1 - sq_norm_xi/sq_xn
    tail_comp_first = tail_comp_tmp1 + (tail_comp_tmp2 + inv_cx2n)*vn/2
    tail_comp_second = tail_comp_tmp1 + (tail_comp_tmp2 - inv_cx2n)*vn/2
    return  torch.cat((other_comps, tail_comp_first, tail_comp_second), dim=-1) 
 

# randomly init x for halfspace
# NOTE: check may fail for lorentz model if last dim too large/too small (too close to 0)
xh = torch.randn(11, 5, 7, dtype=torch.float64)
xh[..., -1].abs_()
xh[..., -1].clamp_min_(20)
assert U.check(xh, c) == True
# no need to check in tan space for halfspace
xh_tan = torch.randn_like(xh)
# get y through expmap
yh = U.expmap(xh, xh_tan, c)
# get tan in lorentz
xl_tan = to_lorentz_tan(xh, xh_tan, c)
xl_tan_prime = U.to_lorentz_tan(xh, xh_tan, c)
assert torch.all(torch.isclose(xl_tan, xl_tan_prime))
xl = U.to_lorentz(xh, c)
# yp transform from yu
yl_from_U = U.to_lorentz(yh, c)
yl_from_expmap = L.expmap(xl, xl_tan, c)
assert torch.all(torch.isclose(yl_from_U, yl_from_expmap))
################################################################
#yp_from_U[0,0], yp_from_expmap[0,0]
yl_from_U[0,0], L.proj(yl_from_U, c)[0,0], L.check(yl_from_U, c)

(tensor([-1.1627e-01, -1.1672e-02,  5.9440e-03, -1.3114e-02, -2.0230e-02,
         -4.5188e-02,  9.4246e+00,  9.4695e+00], dtype=torch.float64),
 tensor([-1.1627e-01, -1.1672e-02,  5.9440e-03, -1.3114e-02, -2.0230e-02,
         -4.5188e-02,  9.4246e+00,  9.4695e+00], dtype=torch.float64),
 tensor(True))

## Halfspace to Poincare (correct)

In [5]:
torch.manual_seed(0)
c = 1.2
# U to B; halfspace to poincare
############################################
def to_poincare_tan(x, v, c):
    sqrt_c = c**0.5
    
    vi, vn = v[..., :-1], v[..., -1].unsqueeze(-1)
    xi, xn = x[..., :-1], x[..., -1].unsqueeze(-1)
    mu_x = 2. / (1 + 2*sqrt_c*xn + c*sq_norm(x))
    sq_mu_x = mu_x.square()
    # ui when 1<=i<=n-1
    other_comps_1 = mu_x*vi
    other_comps_2 = sq_mu_x*c*inner_product(x, v, keepdim=True)*xi
    other_comps_3 = sqrt_c*sq_mu_x*vn*xi
    other_comps = other_comps_1 - other_comps_2 - other_comps_3
    # un
    diff_xnv_vnx = xn*v - vn*x
    inner_tmp = inner_product(x, v+sqrt_c*diff_xnv_vnx)
    tail_comp_1 = sqrt_c*sq_mu_x*inner_tmp
    tail_comp_2 = mu_x*vn*(1-sqrt_c*mu_x*xn)
    tail_comp = tail_comp_1 + tail_comp_2
    #### --------------------------------------------------
    # tail_comp_1 = sqrt_c * sq_mu_x*(1+sqrt_c*xn)*inner_product(x,v)
    # tail_comp_2 = c*sq_mu_x*sq_norm(x)*vn
    # tail_comp_3 = mu_x*vn*(1-sqrt_c*mu_x*xn)
    # tail_comp = tail_comp_1-tail_comp_2+tail_comp_3

    return torch.cat((other_comps, tail_comp), dim=-1)
 

# randomly init x for halfspace
xh = torch.randn(5, 10, 9, dtype=torch.float64)
xh[..., -1].abs_()
xh[..., -1].clamp_min_(1)
assert U.check(xh, c) == True
# no need to check in tan space for halfspace
xh_tan = torch.randn_like(xh)
# get y through expmap
yh = U.expmap(xh, xh_tan, c)
# get tan in poincare
tmp_xl = U.to_lorentz(xh, c)
xp_tan = to_poincare_tan(xh, xh_tan, c) 
tmp_xp_tan = L.to_poincare_tan(tmp_xl, U.to_lorentz_tan(xh, xh_tan, c), c)
tmp_tmp_xp_tan = U.to_poincare_tan(xh, xh_tan, c) 
assert torch.all(torch.isclose(xp_tan, tmp_xp_tan))
assert torch.all(torch.isclose(xp_tan, tmp_tmp_xp_tan))
xp = U.to_poincare(xh, c)
# yp transform from yu
yp_from_U = U.to_poincare(yh, c)
yp_from_expmap = B.expmap(xp, xp_tan, c)
assert torch.all(torch.isclose(yp_from_U, yp_from_expmap))

################################################################
yp_from_U[0,0], yp_from_expmap[0,0]

(tensor([-0.2635, -0.0861, -0.1185,  0.0367, -0.1766, -0.1537, -0.0381, -0.1080,
          0.7943], dtype=torch.float64),
 tensor([-0.2635, -0.0861, -0.1185,  0.0367, -0.1766, -0.1537, -0.0381, -0.1080,
          0.7943], dtype=torch.float64))

## Poincare to Halfspace

In [6]:
torch.manual_seed(0)
c = 1.0
# U to B; halfspace to poincare
############################################
def to_halfspace_tan(x, v, c):
    sqrt_c = c**0.5
    vi, vn = v[..., :-1], v[..., -1].unsqueeze(-1)
    xi, xn = x[..., :-1], x[..., -1].unsqueeze(-1)
    mu_x = 2. / (1 - 2*sqrt_c*xn + c*sq_norm(x))
    sq_mu_x = mu_x.square()
    # ui when 1<=i<=n-1
    other_comps_1 = mu_x*vi
    other_comps_2 = sq_mu_x*c*inner_product(x, v, keepdim=True)*xi
    other_comps_3 = sqrt_c*sq_mu_x*vn*xi
    other_comps = other_comps_1 - other_comps_2 + other_comps_3
    # un
    diff_xnv_vnx = xn*v - vn*x
    inner_tmp = inner_product(x, sqrt_c * diff_xnv_vnx-v)
    tail_comp_1 = sqrt_c * sq_mu_x * inner_tmp
    tail_comp_2 = mu_x * vn * (1 + sqrt_c*mu_x*xn)
    tail_comp = tail_comp_1 + tail_comp_2
    return torch.cat((other_comps, tail_comp), dim=-1)
 
# randomly init x for halfspace
xp = B.proj(torch.randn(5, 7, 9, dtype=torch.float64)/10, c)
assert B.check(xp, c) == True
xp_tan = torch.randn_like(xp)
# get y through expmap
yp = B.expmap(xp, xp_tan, c)
# get tan in halfspace
xh_tan = to_halfspace_tan(xp, xp_tan, c)
tmp_xl = B.to_lorentz(xp, c)
tmp_xh_tan = L.to_halfspace_tan(tmp_xl, B.to_lorentz_tan(xp, xp_tan, c), c)
xh_tan_prime = B.to_halfspace_tan(xp, xp_tan, c)
assert torch.all(torch.isclose(xh_tan, xh_tan_prime))
xh = B.to_halfspace(xp, c)
# yh transform from yp
yh_from_B = B.to_halfspace(yp, c)
yh_from_expmap = U.expmap(xh, xh_tan, c)
assert torch.all(torch.isclose(yh_from_B, yh_from_expmap))


################################################################

#yh_from_B[0,0], yh_from_expmap[0,0]
tmp_xh_tan[0,0], xh_tan[0,0], tmp_xh_tan[0,0]-xh_tan[0,0]

(tensor([ 1.1938, -1.2846, -0.7843, -0.0814,  0.0374,  1.9904, -1.0809, -0.7748,
         -3.5452], dtype=torch.float64),
 tensor([ 1.1938, -1.2846, -0.7843, -0.0814,  0.0374,  1.9904, -1.0809, -0.7748,
         -3.5452], dtype=torch.float64),
 tensor([ 2.2204e-16, -2.2204e-16, -5.5511e-16, -5.5511e-17,  0.0000e+00,
          0.0000e+00, -2.2204e-16, -2.2204e-16,  0.0000e+00],
        dtype=torch.float64))

## Poincare to Lorentz (correct)

In [7]:
torch.manual_seed(0)
c = 1.2
# B to L; poincare to lorentz
############################################
def to_lorentz_tan(x, v, c):
    lambda_x = B._lambda_x(x, c)
    sq_lambda_x = lambda_x.square()
    inner_x_v = inner_product(x,v)
    other_comps_1 = lambda_x * v
    other_comps_2 = c * sq_lambda_x * inner_x_v * x
    other_comps = other_comps_1 + other_comps_2
    tail_comp = c**0.5 * sq_lambda_x * inner_x_v
    return torch.cat((other_comps, tail_comp), dim=-1)

# randomly init x for halfspace
xp = B.proj(torch.randn(5, 7, 9, dtype=torch.float64)/10, c)
assert B.check(xp, c) == True
xp_tan = torch.randn_like(xp)
# get y through expmap
yp = B.expmap(xp, xp_tan, c)
# get tan in lorentz
xl_tan = to_lorentz_tan(xp, xp_tan, c)
xl_tan_prime = B.to_lorentz_tan(xp, xp_tan, c)
assert torch.all(torch.isclose(xl_tan, xl_tan_prime))
xl = B.to_lorentz(xp, c)
# yl transform from yp
yl_from_B = B.to_lorentz(yp, c)
yl_from_expmap = L.expmap(xl, xl_tan, c)
assert torch.all(torch.isclose(yl_from_B, yl_from_expmap))
# ################################################################
yl_from_B[0,0], yl_from_expmap[0,0], L.check(yl_from_expmap, c)

(tensor([-1597.1626,  -913.7385, -1324.6462,   884.0109,  -797.2435,  -290.8969,
         -1053.4376, -1141.5195, -2357.5828,  3822.7244], dtype=torch.float64),
 tensor([-1597.1626,  -913.7385, -1324.6462,   884.0109,  -797.2435,  -290.8969,
         -1053.4376, -1141.5195, -2357.5828,  3822.7244], dtype=torch.float64),
 tensor(True))